<a href="https://colab.research.google.com/github/yr0118kim/MachineLeaningClass/blob/main/0419_%EA%B5%AC%EC%A7%80%EB%A9%B4%EC%9E%91%EC%9D%80%EB%8F%84%EC%84%9C%EA%B4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import gdown
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np # 필요한 라이브러리를 import해옴
yr_book = pd.read_csv('yr_book.csv',low_memory = False,encoding='cp949')

  #NAN인 열을 삭제
yr_book = yr_book.dropna(axis = 1, how = "all")

#대출건수를 합치기 위해 필요한 행만 추출해 count_df 프레임 만들기
#도서명, 저자, ISBN, 권, 대출건수
count_df = yr_book[['도서명','저자','ISBN','권','대출건수']]
#도서명, 저자, ISBN, 권을 기준으로 대출건수를 groupby 하기
loan_count = count_df.groupby(by=['도서명','저자','ISBN','권'], dropna=False).sum()

#원본 데이터 프레임에서 중복된 행을 제외하고 고유한 행만 추출해 복사
dup_rows = yr_book.duplicated(subset=['도서명','저자','ISBN','권'])
unique_rows = ~dup_rows
yr_book3 = yr_book[unique_rows]

#도서명, 저자, ISBN, 권을 인덱스로 설정
yr_book3.set_index(['도서명','저자','ISBN','권'], inplace = True)

#loan_count에 저장된 누적 대출건수를 업데이트
yr_book3.update(loan_count)

#인덱스 재설정
yr_book4 = yr_book3.reset_index()

#원본 데이터프레임의 열 순서를 변경
yr_book4 = yr_book4[yr_book.columns]
  
def yr_book_info(row):
  yr_title = row['도서명']
  yr_author = row['저자']
  yr_pub = row['출판사']
  yr_year = row['발행년도'] # 열값 설정
  
  isbn = row['ISBN']
  url = f'http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}'  #yes24에서 찾고 싶은 책의 ISBN을 가져와 url로 변환

  r = requests.get(url)  # url을 가져온다

  soup = BeautifulSoup(r.text, 'html.parser') #가져온 홈페이지를 beautifulsoup 를 활용해 html로 변환

  try:
    if pd.isna(yr_title): # title이 nan값이라면 soup.find로 title이 있는 클래스 안 값 get.text()로 불러오기
      yr_title = soup.find('a', attrs={'class':'gd_name'}).get_text()
  except AttributeError:
    pass

  try:
    if pd.isna(yr_author): # 만약 author이 nan값이라면 soup.find로 span 태그를 찾아 그 안에 있는 a태그 for문으로 가져오기
      authors = soup.find('span', attrs={'class':'info_auth'}).find_all('a') 
      yr_author = list(tag.get_text() for tag in authors) 
  except AttributeError:
    pass

  try:
    if pd.isna(yr_pub):  # 만약 pub이 nan값이라면 soup.find로 span 태그를 찾아 그 안에 있는 a태그를 get.text()로 가져오기
      yr_pub = soup.find('span', attrs={'class':'info_pub'}).find('a').get_text() 
  except AttributeError:
    pass
  
  try:
    if pd.isna(yr_year): # 만약 이 nan값이라면 soup.find로 span 태그를 찾아 그 안에 있는 a태그를 get.text()로 가져오기
      year_str = soup.find('span', attrs={'class':'info_date'}).get_text() 
      yr_year = re.findall(r'\d{4}', year_str)[0]
  except AttributeError: #에러 있으면 패스하기
    pass
  

  return yr_title,yr_author,yr_pub,yr_year


def yr_book_fixing(yr_book4):

  yr_book4 = yr_book4.astype({'도서권수': 'int32', '대출건수':'int32'}) #도서권수, 대출건수 정수형으로 변환

  set_isbn_na_rows = yr_book4['세트 ISBN'].isna()  # 세트 ISBN값 지우기
  yr_book4.replace({np.nan: ''})
  
  yr_book4.loc[set_isbn_na_rows,"세트 ISBN"] = ""

  yr_book5 = yr_book4.replace({'발행년도': {r'\d{2}(\d{2})'}}, regex=True) #발행년도 값 정규식처리하기
  unkown_year = yr_book5['발행년도'].str.contains('\D', na=True) # 이상한 년도 값 지우기
  yr_book5.loc[unkown_year, '발행년도'] = '-1'

  yr_book5 = yr_book5.astype({"발행년도":"int32"})  #발행년도 값 정수형 변환

  dangun_yy_rows = yr_book5['발행년도'].gt(4000)
  yr_book5.loc[dangun_yy_rows, '발행년도'] = yr_book5.loc[dangun_yy_rows, '발행년도'] - 2333 # 비정상적인 년도값 처리

  dangun_year = yr_book5['발행년도'].gt(4000)
  yr_book5.loc[dangun_year, '발행년도'] = -1

  old_books = yr_book5['발행년도'].gt(0) & yr_book5['발행년도'].lt(1900)
  yr_book5.loc[old_books, '발행년도'] = -1

  na_rows = yr_book5['도서명'].isna() | yr_book5['저자'].isna() | yr_book5['출판사'].isna() | yr_book5['발행년도'].eq(-1) # 처리한 도서명,저자,출판사,발행년도 열에 집어넣기


  
  updated_sample = yr_book5[na_rows].apply(yr_book_info,axis=1)  
  updated_sample.columns = ['도서명','저자','출판사','발행년도']
  yr_book5.update(updated_sample, overwrite = True) # 업데이트

  yr_book6 = yr_book5[yr_book5['발행년도']!=-1]
  yr_book6 = yr_book6[yr_book5['도서명']!=np.nan]
  yr_book6 = yr_book6[yr_book5['저자']!=np.nan]
  yr_book6 = yr_book6[yr_book5['출판사']!=np.nan] # NaN값 아닌 것만 리턴

  return yr_book6
yr_book7 = yr_book_fixing(yr_book4[70:80])
yr_book7

<ipython-input-37-17de0791adf5>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yr_book3.update(loan_count)


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
70,71,하루살이가 만난 내일,나현정 지음,글로연,2023,9788992704717,,7,NaN,813.80,1,0,2023-03-10
71,72,호랭떡집,서현 글·그림,사계절,2023,9791169811163,,7,NaN,813.80,1,2,2023-03-10
72,73,판다 여행사,노하나 하루카 글·그림;송지현 옮김,김영사,2022,9788934940746,,7,NaN,833.80,1,2,2023-03-10
73,74,35년,박시백 글·그림,비아북,2019,9791189426415,9791186712634.0,0,4,911.06,1,1,2023-03-10
74,75,할머니와 봄의 정원,강혜영 글·그림,팜파스,2021,9791170264026,,7,NaN,813.80,1,0,2023-03-10
75,76,파리왕국 모기대왕,이은하 지음;고윤아 그림,소라피리,2023,9791192682013,,7,11,813.80,1,1,2023-03-10
76,77,지구가 보내온 편지,몰리 블룸;마크 산체스;샌든 토튼 [공]글;마이크 오르단 그림;이유림 옮김,에듀앤테크,2022,9791167290366,,7,NaN,843.00,1,0,2023-03-10
77,78,잠이 솔솔 핫초코,양선 지음,소원나무,2022,9791170441595,,7,NaN,813.80,1,0,2023-03-10
78,79,큰 딸기가 말했어,우은선 글;낙서쟁이 김선생 그림,걸음동무,2023,9791190898911,,7,NaN,813.80,1,1,2023-03-10
79,80,"조금만, 조금만 더",소연정 글·그림,모든요일그림책,2023,9788925577180,,7,NaN,813.80,1,0,2023-03-10
